In [85]:
!pip install transformers
!pip install rouge-score nltk bert-score

In [86]:
import json
from transformers import pipeline
from google.colab import files
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score

In [87]:
# Subir el archivo
uploaded = files.upload()

Saving cleaned_json.json to cleaned_json (6).json


In [88]:
# Asumir que el archivo subido se llama 'input.json'
input_file = list(uploaded.keys())[0]

In [89]:
# Cargar el conjunto de datos desde el archivo JSON subido
with open(input_file, "r") as f:
    data = json.load(f)

In [90]:
# Verificar las claves disponibles en las entradas
print("Available keys in each entry:", data["Sheet1"][0].keys())

Available keys in each entry: dict_keys(['Document', 'Content '])


In [91]:
# Ajustar la clave según la estructura correcta del archivo JSON
content_key = "Content "  # Ajustar según la clave correcta en tu archivo

In [92]:
# Cargar los modelos pre-entrenados para la generación y respuesta de preguntas
question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qg-hl")
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [93]:
# Inicializar el evaluador de Rouge
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [94]:
output_data = []

In [95]:
# Generar preguntas y respuestas
for entry in data["Sheet1"]:
    # Verificar si la clave existe en la entrada
    if content_key not in entry:
        print(f"Key '{content_key}' not found in entry: {entry}")
        continue

    context = entry[content_key]
    print(f"Processing context: {context[:100]}...")  # Mostrar los primeros 100 caracteres del contexto

    try:
        # Generar dos preguntas distintas
        questions = question_generator(f"generate question: {context}", num_return_sequences=2, num_beams=4, max_length=50)
        generated_questions = [q['generated_text'] for q in questions]
    except Exception as e:
        print(f"Error generating questions: {e}")
        continue

    for generated_question in generated_questions:
        print(f"Generated question: {generated_question}")

        # Asegurarse de que la pregunta generada esté completa
        if not generated_question.endswith('?'):
            generated_question += "?"

        try:
            # Generar respuesta con un límite de longitud mayor
            generated_answer = qa_pipeline(question=generated_question, context=context, max_answer_len=200)['answer']
            print(f"Generated answer: {generated_answer}")
        except Exception as e:
            print(f"Error generating answer: {e}")
            continue

        # Filtrar métricas no deseadas

        output_data.append({
            "prompt": generated_question,
            "response": generated_answer,
        })

Token indices sequence length is longer than the specified maximum sequence length for this model (735 > 512). Running this sequence through the model will result in indexing errors


Processing context: There are seven licensed nuclear power plant (NPP) sites in Canada, operated by four different  lice...
Generated question: What is the CNSC's report on the safety of Canadian nuclear power plants?
Generated answer: Integrated Safety Assessment
Generated question: What was the CNSC's report on the safety of Canadian nuclear power plants?
Generated answer: Integrated Safety Assessment of Canadian  Nuclear Power Plants
Processing context: Executive Summary  This report, entitled CNSC Staff Integrated Safety Assessment of Canadian Nuclear...
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants for 2010?
Generated answer: 2010 NPP Report
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants report?
Generated answer: 2010 NPP Report
Processing context: Each year, the Canadian Nuclear Safety Commission (CNSC) publishes a report on the  safety performan...
Generated question: Wha

In [96]:
# Guardar en un archivo JSON
output_file = "qa_pairs.json"
with open(output_file, "w") as f:
    json.dump(output_data, f, indent=4)

In [97]:
print(f"Output saved to {output_file}")

# Descargar el archivo de salida
files.download(output_file)

Output saved to qa_pairs.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>